In [21]:
import numpy as np 
import pandas as pd 
import os

from numpy.random import seed

from keras.models import load_model
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from nltk.corpus import stopwords


In [2]:
train_data = pd.read_csv("final_train.csv")
train_data.head()

,class,desc,rating
0,173,"""Humira gave me my life back. I was sicker th...",10.0
1,533,"""I&#039;m having a terrible time with the hysi...",1.0
2,95,"""I have been on four different birth control p...",10.0
3,560,"""Pain relief is limited and barely noticed. I ...",4.0
4,103,"""Aside from the out of pocket cost ($130.00 fo...",9.0


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Rahul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#REMOVE STOPWORDS

stop = stopwords.words('english')
train_data['desc'] = train_data['desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [5]:
seed(1)

In [6]:

train_data.head()

,class,desc,rating
0,173,"""Humira gave life back. I sicker I ever first ...",10.0
1,533,"""I&#039;m terrible time hysingla. The 40mg str...",1.0
2,95,"""I four different birth control pills past 2 y...",10.0
3,560,"""Pain relief limited barely noticed. I take 18...",4.0
4,103,"""Aside pocket cost ($130.00 me) I would recomm...",9.0


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159523 entries, 0 to 159522
Data columns (total 3 columns):
class     159523 non-null int64
desc      159523 non-null object
rating    159523 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.7+ MB


In [8]:
num_words = 5000

max_length_of_text = 250
batch_size = 40

In [9]:

tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True ,split=' ')
tokenizer.fit_on_texts(train_data['desc'].values)
X = tokenizer.texts_to_sequences(train_data['desc'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(X, maxlen=max_length_of_text)

print(word_index)

Found 49825 unique tokens.
{'i': 1, '039': 2, 'it': 3, 't': 4, 'day': 5, 'm': 6, 'side': 7, 'taking': 8, 've': 9, 's': 10, 'years': 11, 'pain': 12, 'effects': 13, 'first': 14, 'my': 15, 'take': 16, 'the': 17, '2': 18, 'months': 19, 'started': 20, 'like': 21, 'get': 22, 'days': 23, 'time': 24, '3': 25, 'feel': 26, 'would': 27, 'this': 28, 'back': 29, 'pill': 30, 'one': 31, 'weeks': 32, 'week': 33, 'took': 34, 'doctor': 35, 'also': 36, 'period': 37, 'medication': 38, 'weight': 39, 'got': 40, 'month': 41, 'me': 42, 'life': 43, 'since': 44, 'don': 45, 'bad': 46, 'still': 47, 'anxiety': 48, '5': 49, 'really': 50, 'much': 51, 'never': 52, 'better': 53, 'went': 54, '4': 55, 'medicine': 56, 'now': 57, 'work': 58, 'two': 59, 'quot': 60, 'felt': 61, 'well': 62, 'control': 63, 'every': 64, 'good': 65, 'didn': 66, 'year': 67, 'even': 68, 'great': 69, 'night': 70, '1': 71, 'go': 72, 'sleep': 73, 'ago': 74, 'tried': 75, '6': 76, 'after': 77, 'works': 78, 'acne': 79, '10': 80, 'birth': 81, 'made': 82

In [10]:
y = train_data['rating']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(127618, 250) (127618,)
(31905, 250) (31905,)


In [11]:
y_test = pd.get_dummies(y_test)
y_train = pd.get_dummies(y_train)
X_train = np.array(X_train)
X_test = np.array(X_test)


In [12]:
y_train.shape
#we need to remove the extra column now

(127618, 10)

In [13]:
y_train


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
18670,0,0,0,0,0,0,0,0,0,1
4327,0,0,0,0,0,0,0,0,1,0
153572,0,0,0,1,0,0,0,0,0,0
150640,0,0,0,1,0,0,0,0,0,0
128873,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
73349,0,0,0,0,0,0,0,0,1,0
109259,0,0,0,0,0,0,0,0,0,1
50057,0,0,0,0,0,0,1,0,0,0
5192,0,0,0,0,0,0,0,0,1,0


### Model 2: ( Removed Model1 from file because it didn't perform well)


In [14]:
#num_words = 200
#embed_dim = 40 #Change to observe effects,initially 50
#lstm_out_1 = 100 #Change to observe effects
batch_size = 40
#length_of_text = 200
#max_length_of_text = 200

In [15]:
inputs = Input((max_length_of_text, ))
x = Embedding(num_words, 64)(inputs)    
x = LSTM(128,return_sequences = False,dropout=0.2, recurrent_dropout=0.2)(x) 
x = Dense(10,activation='sigmoid')(x)
model2 = Model(inputs, x)
print(model2.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 250, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 420,106
Trainable params: 420,106
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
#needs to be used to continue from previous submission
#model2 = load_model('model_4.h5')

In [18]:
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [34]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = batch_size, epochs = 1) # total epochs = 10

Train on 127618 samples, validate on 31905 samples
Epoch 1/1
127618/127618 [==============================] - 1008s 8ms/step - loss: 1.7427 - acc: 0.3912 - val_loss: 1.6323 - val_acc: 0.4132


In [35]:
model2.fit(X_train, y_train, batch_size = batch_size, epochs = 2) 
#19 epochs, stopped at just 2 because forgot to use validation data,included in remaining 17 epochs

Epoch 1/19
127618/127618 [==============================] - 949s 7ms/step - loss: 1.5751 - acc: 0.4334
Epoch 2/19
127618/127618 [==============================] - 1021s 8ms/step - loss: 1.5193 - acc: 0.4483
Epoch 3/19
   120/127618 [..............................] - ETA: 19:51 - loss: 1.5245 - acc: 0.4333

KeyboardInterrupt: 

In [36]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = batch_size, epochs = 17) # total epochs = 20 # total epochs = 10

Train on 127618 samples, validate on 31905 samples
Epoch 1/17
127618/127618 [==============================] - 1023s 8ms/step - loss: 1.4692 - acc: 0.4635 - val_loss: 1.5512 - val_acc: 0.4367
Epoch 2/17
127618/127618 [==============================] - 1012s 8ms/step - loss: 1.4188 - acc: 0.4810 - val_loss: 1.5332 - val_acc: 0.4479
Epoch 3/17
127618/127618 [==============================] - 1009s 8ms/step - loss: 1.3741 - acc: 0.4968 - val_loss: 1.5288 - val_acc: 0.4530
Epoch 4/17
127618/127618 [==============================] - 975s 8ms/step - loss: 1.3311 - acc: 0.5109 - val_loss: 1.5235 - val_acc: 0.4559
Epoch 5/17
127618/127618 [==============================] - 979s 8ms/step - loss: 1.2924 - acc: 0.5250 - val_loss: 1.5289 - val_acc: 0.4589
Epoch 6/17
127618/127618 [==============================] - 977s 8ms/step - loss: 1.2545 - acc: 0.5384 - val_loss: 1.5281 - val_acc: 0.4619
Epoch 7/17
127618/127618 [==============================] - 976s 8ms/step - loss: 1.2202 - acc: 0.5516 - v

In [42]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = batch_size, epochs = 5) # total epochs = 25 

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 1009s 8ms/step - loss: 0.9556 - acc: 0.6552 - val_loss: 1.5675 - val_acc: 0.5198
Epoch 2/5
127618/127618 [==============================] - 983s 8ms/step - loss: 0.9403 - acc: 0.6616 - val_loss: 1.5628 - val_acc: 0.5249
Epoch 3/5
127618/127618 [==============================] - 984s 8ms/step - loss: 0.9243 - acc: 0.6664 - val_loss: 1.5857 - val_acc: 0.5260
Epoch 4/5
127618/127618 [==============================] - 990s 8ms/step - loss: 0.9110 - acc: 0.6727 - val_loss: 1.5784 - val_acc: 0.5292
Epoch 5/5
127618/127618 [==============================] - 983s 8ms/step - loss: 0.8969 - acc: 0.6781 - val_loss: 1.5764 - val_acc: 0.5331


In [23]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = batch_size, epochs = 5) # total epochs = 30 (for submission 5) 

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 1024s 8ms/step - loss: 0.8871 - acc: 0.6812 - val_loss: 1.5791 - val_acc: 0.5336
Epoch 2/5
127618/127618 [==============================] - 993s 8ms/step - loss: 0.8735 - acc: 0.6856 - val_loss: 1.5975 - val_acc: 0.5372
Epoch 3/5
127618/127618 [==============================] - 988s 8ms/step - loss: 0.8631 - acc: 0.6915 - val_loss: 1.5869 - val_acc: 0.5391
Epoch 4/5
127618/127618 [==============================] - 996s 8ms/step - loss: 0.8531 - acc: 0.6936 - val_loss: 1.5987 - val_acc: 0.5443
Epoch 5/5
127618/127618 [==============================] - 1014s 8ms/step - loss: 0.8460 - acc: 0.6968 - val_loss: 1.6013 - val_acc: 0.5486


In [27]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 300, epochs = 5) 
# total epochs = 35 (for submission 6), changed batch size from 40 to 300

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 972s 8ms/step - loss: 0.7762 - acc: 0.7236 - val_loss: 1.6065 - val_acc: 0.5528
Epoch 2/5
127618/127618 [==============================] - 860s 7ms/step - loss: 0.7637 - acc: 0.7284 - val_loss: 1.6148 - val_acc: 0.5571
Epoch 3/5
127618/127618 [==============================] - 869s 7ms/step - loss: 0.7540 - acc: 0.7319 - val_loss: 1.6172 - val_acc: 0.5591
Epoch 4/5
127618/127618 [==============================] - 704s 6ms/step - loss: 0.7491 - acc: 0.7321 - val_loss: 1.6238 - val_acc: 0.5595
Epoch 5/5
127618/127618 [==============================] - 676s 5ms/step - loss: 0.7426 - acc: 0.7356 - val_loss: 1.6224 - val_acc: 0.5648


In [31]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 300, epochs = 5) 
# total epochs = 40 (for submission 7)

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 704s 6ms/step - loss: 0.7355 - acc: 0.7383 - val_loss: 1.6301 - val_acc: 0.5651
Epoch 2/5
127618/127618 [==============================] - 678s 5ms/step - loss: 0.7323 - acc: 0.7396 - val_loss: 1.6332 - val_acc: 0.5677
Epoch 3/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.7275 - acc: 0.7397 - val_loss: 1.6391 - val_acc: 0.5672
Epoch 4/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.7240 - acc: 0.7415 - val_loss: 1.6427 - val_acc: 0.5692
Epoch 5/5
127618/127618 [==============================] - 675s 5ms/step - loss: 0.7172 - acc: 0.7431 - val_loss: 1.6446 - val_acc: 0.5714


In [33]:
score,acc = model2.evaluate(X_test, y_test, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

31905/31905 [==============================] - 47s 1ms/step
Score: 1.64
Validation Accuracy: 0.57


In [34]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 300, epochs = 5) 
# total epochs = 45 

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.7192 - acc: 0.7419 - val_loss: 1.6445 - val_acc: 0.5724
Epoch 2/5
127618/127618 [==============================] - 682s 5ms/step - loss: 0.7092 - acc: 0.7467 - val_loss: 1.6533 - val_acc: 0.5748
Epoch 3/5
127618/127618 [==============================] - 681s 5ms/step - loss: 0.7081 - acc: 0.7478 - val_loss: 1.6473 - val_acc: 0.5751
Epoch 4/5
127618/127618 [==============================] - 684s 5ms/step - loss: 0.7044 - acc: 0.7480 - val_loss: 1.6574 - val_acc: 0.5777
Epoch 5/5
127618/127618 [==============================] - 683s 5ms/step - loss: 0.6975 - acc: 0.7507 - val_loss: 1.6650 - val_acc: 0.5769


In [36]:
score,acc = model2.evaluate(X_test, y_test, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

31905/31905 [==============================] - 47s 1ms/step
Score: 1.67
Validation Accuracy: 0.58


In [37]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 300, epochs = 5) 
# total epochs = 50 (for submission 9) - BEST

Train on 127618 samples, validate on 31905 samples
Epoch 1/5
127618/127618 [==============================] - 683s 5ms/step - loss: 0.6931 - acc: 0.7514 - val_loss: 1.6645 - val_acc: 0.5779
Epoch 2/5
127618/127618 [==============================] - 683s 5ms/step - loss: 0.6921 - acc: 0.7515 - val_loss: 1.6664 - val_acc: 0.5809
Epoch 3/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.6895 - acc: 0.7545 - val_loss: 1.6703 - val_acc: 0.5801
Epoch 4/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.6826 - acc: 0.7556 - val_loss: 1.6742 - val_acc: 0.5836
Epoch 5/5
127618/127618 [==============================] - 679s 5ms/step - loss: 0.6804 - acc: 0.7563 - val_loss: 1.6671 - val_acc: 0.5846


In [38]:
model2.save('model-2017A7PS0050G.h5')
#Total epochs run = 50

In [39]:
score,acc = model2.evaluate(X_test, y_test, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

31905/31905 [==============================] - 46s 1ms/step
Score: 1.67
Validation Accuracy: 0.58
